In [1]:
import pandas as pd

import numpy as np

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(context = 'notebook', #mostly controls relative sizes of things on plot 
#                         #The base context is “notebook”, and the other contexts are “paper”, “talk”, and “poster”
#         style = 'darkgrid', #dict, None, or one of {darkgrid, whitegrid, dark, white, ticks}
#         palette = 'deep', # Should be something that color_palette() can process.
#         font_scale = 1, 
#         color_codes = False, 
#         rc = None)

# from IPython.core.interactiveshell import InteractiveShell 
# InteractiveShell.ast_node_interactivity = 'last_expr' 
# setting = "all" allows multiple outputs to be displayed for a given input cell. don't use w plotting!

from IPython.display import display

# %matplotlib notebook
#%matplotlib inline

pd.__version__ , np.__version__ #,  matplotlib.__version__, sns.__version__

('0.23.0', '1.14.3')

In [2]:
from sklearn.model_selection import train_test_split, learning_curve, cross_val_score, KFold, StratifiedKFold, \
                                    ShuffleSplit, GridSearchCV, RandomizedSearchCV

from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier

In [3]:
pwd

'/home/thedohn/Notebooks'

In [4]:
#cd '/Users/DonBunk/Desktop/Google Drive/data_science/Python_Projects/Home_Credit_Default_Risk/'

In [5]:
from Home_Credit_package.master_pipeline import master_pipeline

# load this

## load df.

In [6]:
path_o = ''#'wrangling/create_final_wrangled_csv/'
path_input = ''#'level_3_ensembling/'
original_cleaned_df = pd.read_csv(path_o + 'complete_initial_wrangled_data.csv', index_col = 'SK_ID_CURR')
level_1_metafeatures_df = pd.read_csv(path_input + 'level_3_input_FINAL_level_2_meta_features_df.csv', index_col = 'SK_ID_CURR')
total_df = pd.merge(original_cleaned_df, level_1_metafeatures_df, left_index=True, right_index=True, how = 'outer' )

In [7]:
total_df.info(verbose = True, null_counts = True);

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 100002 to 456255
Data columns (total 92 columns):
TARGET                                307511 non-null int64
NAME_CONTRACT_TYPE                    307511 non-null object
CODE_GENDER                           307511 non-null object
FLAG_OWN_CAR                          307511 non-null object
FLAG_OWN_REALTY                       307511 non-null object
CNT_CHILDREN                          307511 non-null int64
NAME_EDUCATION_TYPE                   307511 non-null object
NAME_FAMILY_STATUS                    307511 non-null object
DAYS_BIRTH                            307511 non-null int64
FLAG_EMP_PHONE                        307511 non-null int64
FLAG_WORK_PHONE                       307511 non-null int64
FLAG_PHONE                            307511 non-null int64
FLAG_EMAIL                            307511 non-null int64
CNT_FAM_MEMBERS                       307511 non-null int64
REGION_RATING_CLIENT                  

In [8]:
# CHECK: this should be empty if everything is non null
total_df.isnull().any()[total_df.isnull().any()==True]

Series([], dtype: bool)

# models with only EXT SOURCES + level 2 final scores.                                     

In [56]:
minimal_feats = ['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3',
                 'pwr_rescale_RanFor_EXTpoly_Level2',     
                    'pwr_rescale_LogReg_EXTpoly_Level2',     
                    'pwr_rescale_MLP_EXTpoly_Level2',        
                    'pwr_rescale_RanFor_AllFeats_Level2', 
                    'pwr_rescale_LogReg_AllFeats_Level2',  
                    'pwr_rescale_MLP_AllFeats_Level2',
                    ]

In [57]:
total_df_piped, final_feature_list, total_pipeline, trans_list = master_pipeline(df_in = total_df[minimal_feats],
                                                                                 int_cutoff=20, 
                                                                                 poly_deg=4, 
                                                                                 feats_with_interaction=[]
                                                                                 )

Total original feat len is 9. Sum of feats is 9.
Final array is length 45. final feature list is length 45.


### DONE. AdaBoost + log reg grid search.

In [58]:
param_grid = [{ 'base_estimator__C': [300,400,500,600,700,800,900,1000],
                'learning_rate':       [.1,.5,1,2],
              }]

my_LgRg_4_Ada = LogisticRegression(penalty= 'l2', 
                                   random_state = 0, 
                                   class_weight = None) 

my_AdaBC_LogReg_minimal_feats = AdaBoostClassifier(base_estimator = my_LgRg_4_Ada, 
                                                   n_estimators = 50, 
                                                   algorithm = 'SAMME.R', 
                                                   random_state = 0)

my_AdaBC_LogReg_minimal_feats_grid_search = GridSearchCV(my_AdaBC_LogReg_minimal_feats, 
                                                       param_grid, 
                                                       cv = 3, # for integer inputs, Stratified folding is default. 
                                                       return_train_score = True , 
                                                       scoring = 'roc_auc',
                                                       n_jobs = -1, 
                                                       refit = True,
                                                       verbose = 51,
                                                        )

my_AdaBC_LogReg_minimal_feats_grid_search.fit(total_df_piped, 
                                                total_df['TARGET'])

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] base_estimator__C=300, learning_rate=0.1 ........................
Memmaping (shape=(307511, 45), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_1824_139943438208080/1824-139943440924744-1893222a0eaf16b127df7344cdf2dd31.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_1824_139943438208080/1824-139943440924744-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_1824_139943438208080/1824-139943440924744-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_1824_139943438208080/1824-139943440924744-981b55e712abb5a5a2369212f2856f0a.pkl
[CV] base_estimator__C=300, learning_rate=0.1 ........................
Memmaping (shape=(205007,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_1824_139943438208080/1824-139943440924744-52b45483e9e8ae

[CV]  base_estimator__C=500, learning_rate=0.5, score=0.7774538571802507, total= 3.1min
[CV] base_estimator__C=600, learning_rate=0.1 ........................
[CV]  base_estimator__C=500, learning_rate=1, score=0.7724768883569914, total= 2.5min
[Parallel(n_jobs=-1)]: Done  31 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed: 12.4min
[CV] base_estimator__C=600, learning_rate=0.5 ........................
[CV]  base_estimator__C=500, learning_rate=1, score=0.7774557206297972, total= 2.5min
[CV] base_estimator__C=600, learning_rate=0.5 ........................
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 13.7min
[CV]  base_estimator__C=500, learning_rate=2, score=0.7708966319404624, total= 2.7min
[CV] base_estimator__C=600, learning_rate=0.5 ........................
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 14.1min
[CV]  base_estimator__C=500, learning_rate=2, score=0.7701013001077692, total= 2.8min
[CV] base_estimator__C=600, learning

[CV] base_estimator__C=900, learning_rate=0.5 ........................
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 27.3min
[CV]  base_estimator__C=800, learning_rate=1, score=0.777460831329345, total= 2.7min
[CV] base_estimator__C=900, learning_rate=0.5 ........................
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 28.1min
[CV]  base_estimator__C=800, learning_rate=2, score=0.7684496524007737, total= 2.7min
[CV] base_estimator__C=900, learning_rate=0.5 ........................
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed: 28.6min
[CV]  base_estimator__C=800, learning_rate=2, score=0.767893683444385, total= 2.8min
[CV] base_estimator__C=900, learning_rate=1 ..........................
[Parallel(n_jobs=-1)]: Done  71 tasks      | elapsed: 28.9min
[CV]  base_estimator__C=800, learning_rate=2, score=0.7735962178001882, total= 2.8min
[CV] base_estimator__C=900, learning_rate=1 ..........................
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed: 2

GridSearchCV(cv=3, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'base_estimator__C': [300, 400, 500, 600, 700, 800, 900, 1000], 'learning_rate': [0.1, 0.5, 1, 2]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [59]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_AdaBC_LogReg_minimal_feats_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'base_estimator__C': 300, 'learning_rate': 0.1}",0.774639
1,"{'base_estimator__C': 300, 'learning_rate': 0.5}",0.774665
2,"{'base_estimator__C': 300, 'learning_rate': 1}",0.774668
3,"{'base_estimator__C': 300, 'learning_rate': 2}",0.773671
4,"{'base_estimator__C': 400, 'learning_rate': 0.1}",0.774649
5,"{'base_estimator__C': 400, 'learning_rate': 0.5}",0.774666
6,"{'base_estimator__C': 400, 'learning_rate': 1}",0.774670
7,"{'base_estimator__C': 400, 'learning_rate': 2}",0.773025
8,"{'base_estimator__C': 500, 'learning_rate': 0.1}",0.774655
9,"{'base_estimator__C': 500, 'learning_rate': 0.5}",0.774668


In [60]:
my_AdaBC_LogReg_minimal_feats_grid_search.best_score_

0.7746843824884295

In [61]:
LgRg_optimized_parms = my_AdaBC_LogReg_minimal_feats_grid_search.best_params_
LgRg_optimized_parms

{'base_estimator__C': 1000, 'learning_rate': 1}

In [62]:
train_scores = [x[1] for x in my_AdaBC_LogReg_minimal_feats_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7749187198256312

In [63]:
del my_AdaBC_LogReg_minimal_feats_grid_search

# models with all features

In [9]:
feats_to_drop = ['pwr_rescale_RanFor_EXTpoly_Level2',     
                 'pwr_rescale_LogReg_EXTpoly_Level2',     
                 'pwr_rescale_MLP_EXTpoly_Level2',        
                ]

In [10]:
total_df_piped, final_feature_list, total_pipeline, trans_list = \
master_pipeline(df_in = total_df.drop(columns = feats_to_drop),
               int_cutoff=20, 
               poly_deg=4, 
               feats_with_interaction=[]
               )

Total original feat len is 88. Sum of feats is 88.
Final array is length 432. final feature list is length 432.


### Adaboost + log reg grid search

In [16]:
param_grid = [{ 'base_estimator__C': [100,200,300,500] ,
                'learning_rate':       [.001,.01,.1]}]

my_LgRg_4_Ada = LogisticRegression(penalty= 'l2', 
                                   random_state = 0, 
                                   class_weight = None) 

my_AdaBC_LogReg_all_feats = AdaBoostClassifier(base_estimator = my_LgRg_4_Ada, 
                                               n_estimators = 50, 
                                               algorithm = 'SAMME.R', 
                                               random_state = 0)

my_AdaBC_LogReg_all_feats_grid_search = GridSearchCV(my_AdaBC_LogReg_all_feats, 
                                                       param_grid, 
                                                       cv = 3, # for integer inputs, Stratified folding is default. 
                                                       return_train_score = True , 
                                                       scoring = 'roc_auc',
                                                       n_jobs = -1, 
                                                       refit = True,
                                                       verbose = 51,
                                                        )

my_AdaBC_LogReg_all_feats_grid_search.fit(total_df_piped, 
                                           total_df['TARGET'])

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Memmaping (shape=(76877750,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_2392_139946017526224/2392-139946013753072-daf169b31db8d878d50c1206f310810c.pkl
Memmaping (shape=(307512,), dtype=int32) to new file /dev/shm/joblib_memmaping_pool_2392_139946017526224/2392-139946013753072-d16d7957d6b5ec4281598ec2ca74d159.pkl
Memmaping (shape=(76877750,), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_2392_139946017526224/2392-139946013753072-9e9de26c8aaed60aa6668a1090e2249b.pkl
[CV] base_estimator__C=100, learning_rate=0.001 ......................
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_2392_139946017526224/2392-139946013753072-36f41ebc26c9047822d0779f2962f9e3.pkl
Memmaping (shape=(307511,), dtype=int64) to new file /dev/shm/joblib_memmaping_pool_2392_139946017526224/2392-139946013753072-7fee98903a11851f0c36b343c7ef5338.pkl
Memmaping (shape=(205007,), dtype=int64) to new

[Parallel(n_jobs=-1)]: Done  28 out of  36 | elapsed: 181.6min remaining: 51.9min
[CV]  base_estimator__C=300, learning_rate=0.1, score=0.7710484405494245, total=72.7min
[Parallel(n_jobs=-1)]: Done  29 out of  36 | elapsed: 184.5min remaining: 44.5min
[CV]  base_estimator__C=300, learning_rate=0.1, score=0.7750409529268022, total=69.7min
[Parallel(n_jobs=-1)]: Done  30 out of  36 | elapsed: 186.0min remaining: 37.2min
[CV]  base_estimator__C=500, learning_rate=0.01, score=0.7719890519332939, total=54.7min
[Parallel(n_jobs=-1)]: Done  31 out of  36 | elapsed: 190.3min remaining: 30.7min
[CV]  base_estimator__C=500, learning_rate=0.01, score=0.7702227526378096, total=52.4min
[Parallel(n_jobs=-1)]: Done  32 out of  36 | elapsed: 206.4min remaining: 25.8min
[CV]  base_estimator__C=500, learning_rate=0.01, score=0.7753933654783723, total=44.8min
[Parallel(n_jobs=-1)]: Done  33 out of  36 | elapsed: 216.6min remaining: 19.7min
[CV]  base_estimator__C=500, learning_rate=0.1, score=0.769728248

GridSearchCV(cv=3, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          learning_rate=1.0, n_estimators=50, random_state=0),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'base_estimator__C': [100, 200, 300, 500], 'learning_rate': [0.001, 0.01, 0.1]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=51)

In [17]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(my_AdaBC_LogReg_all_feats_grid_search.cv_results_)[['params','mean_test_score']]

,params,mean_test_score
0,"{'base_estimator__C': 100, 'learning_rate': 0.001}",0.772182
1,"{'base_estimator__C': 100, 'learning_rate': 0.01}",0.772491
2,"{'base_estimator__C': 100, 'learning_rate': 0.1}",0.772173
3,"{'base_estimator__C': 200, 'learning_rate': 0.001}",0.772510
4,"{'base_estimator__C': 200, 'learning_rate': 0.01}",0.772603
5,"{'base_estimator__C': 200, 'learning_rate': 0.1}",0.772037
6,"{'base_estimator__C': 300, 'learning_rate': 0.001}",0.772573
7,"{'base_estimator__C': 300, 'learning_rate': 0.01}",0.772590
8,"{'base_estimator__C': 300, 'learning_rate': 0.1}",0.771961
9,"{'base_estimator__C': 500, 'learning_rate': 0.001}",0.772571


In [18]:
my_AdaBC_LogReg_all_feats_grid_search.best_score_

0.7726025504592231

In [19]:
LgRg_optimized_parms = my_AdaBC_LogReg_all_feats_grid_search.best_params_
LgRg_optimized_parms

{'base_estimator__C': 200, 'learning_rate': 0.01}

In [20]:
train_scores = [x[1] for x in my_AdaBC_LogReg_all_feats_grid_search.predict_proba(total_df_piped)]
roc_auc_score(total_df['TARGET'], train_scores)

0.7743676835818082

In [21]:
del my_AdaBC_LogReg_all_feats_grid_search